In [1]:
from pyautogui import *
import cv2
import numpy as np 
from tensorflow import keras

ModuleNotFoundError: ignored

In [2]:
model = keras.models.load_model("withdropout.h5")

NameError: ignored

In [ ]:
def getMask(roi, fgbg):
    fgmask=fgbg.apply(roi,learningRate=0)
    kernel=np.ones((5,5),np.uint8)
    morphopen=cv2.morphologyEx(fgmask,cv2.MORPH_OPEN,kernel)
    morphclose=cv2.morphologyEx(morphopen,cv2.MORPH_CLOSE,kernel)
    ret,thresh=cv2.threshold(morphclose,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    return thresh

def main():
    cap=cv2.VideoCapture(0)
    mfound=False
    command = "stationary1"
    gestures = ["up","right","down","select","stationary1","stationary2"] # withdown.h5 deepernnwithoutleft.h5
    marker  = [0,0]
    selection = "Password : "
    back_end = [[1,2,3],[4,5,6],[7,8,9],["Stop",0,"Back"]]
    templist=[0,0,0,0,0,0,0,0,0,0]
    new_command='stationary1'

    while(True):
        ret, frame = cap.read()
        roi=frame[150:400,450:620]
        cv2.rectangle(frame, (450,150), (620,400), (255,255,255))

        if mfound:
            mask=getMask(roi, fgbg)
            mask = cv2.GaussianBlur(mask, (3,3), 20)
            cv2.imshow("mask", mask)
            target = np.stack((mask,) * 3, axis=-1)
            target = target.reshape(1, 250, 170, 3)
            prediction=model.predict_classes(target)    
            for p in range(1,len(templist)):
                templist[p-1]=templist[p]
                templist[len(templist)-1]=prediction[0]
            if templist[0]==templist[1]==templist[2]==templist[3]==templist[4]==templist[5]==templist[6]==templist[7]==templist[8]==templist[9]:
                 new_command = gestures[prediction[0]]

            if command == new_command:
                pass
            else: 
                command = new_command
                if command =="up":
                    press('up')
                elif command =="down":
                    press('down')
                elif command == "right":
                    press('right')
                elif command =="select":
                    press('enter')
                elif command=="left":
                    press('left')
            cv2.putText(frame, command, (0,300), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,255), 2)
        cv2.imshow("frame", frame)

        inp=cv2.waitKey(50)
        if inp==27:
            cap.release()
            cv2.destroyAllWindows()
            break
        elif inp==ord('a'):
            fgbg=cv2.createBackgroundSubtractorMOG2(0,50)
            mfound=True
            
main()